# Async API for Agent

LangChain provides async support for Agents by leveraging the [asyncio](https://docs.python.org/3/library/asyncio.html) library.

Async methods are currently supported for the following `Tool`s: `SerpAPIWrapper` and `LLMMathChain`. Async support for other agent tools are on the roadmap.

For `Tool`s that have `coroutine` implemented (the two mentioned above), the `AgentExecutor` will call `await` them directly. Otherwise, the `AgentExecutor` will call the `Tool`'s `func` via `asyncio.get_event_loop().run_in_executor`.

You can use `arun` to call an `AgentExecutor` asynchronously.

In [2]:
import asyncio
import time

from langchain.agents import initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.tracers import LangChainTracer
from aiohttp import ClientSession

QUESTION = "Who won the US Open men's final in 2019? What is his age raised to the 0.334 power?"

In [3]:
def generate_serially():
    for _ in range(5):
        llm = OpenAI(temperature=0)
        tools = load_tools(["llm-math", "serpapi"], llm=llm)
        agent = initialize_agent(
            tools, llm, agent="zero-shot-react-description", verbose=True
        )
        agent.run(QUESTION)

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print(f"Serial executed in {elapsed:0.2f} seconds.")



> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner"
Observation: Rafael Nadal
Thought: I need to find out Rafael Nadal's age
Action: Search
Action Input: "Rafael Nadal age"
Observation: 36 years
Thought: I need to calculate 36 raised to the 0.334 power
Action: Calculator
Action Input: 36^0.334
Observation: Answer: 3.3098250249682484

Thought: I now know the final answer
Final Answer: Rafael Nadal, aged 36, won the US Open men's final in 2019 and his age raised to the 0.334 power is 3.3098250249682484.

> Finished chain.


> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner"
Observation: Rafael Nadal
Thought: I need to find out Rafael Nadal's age
Action: Search
Acti

In [4]:
async def generate_concurrently():
    agents = []
    # To make async requests in Tools more efficient, you can pass in your own aiohttp.ClientSession, 
    # but you must manually close the client session at the end of your program/event loop
    aiosession = ClientSession()
    for color in ["blue", "green", "red", "pink", "yellow"]:
        # Use a custom CallbackManager to print in different colors.
        manager = CallbackManager([StdOutCallbackHandler(color=color)])
        llm = OpenAI(temperature=0, callback_manager=manager)
        async_tools = load_tools(["llm-math", "serpapi"], llm=llm, aiosession=aiosession)
        agents.append(
            initialize_agent(async_tools, llm, agent="zero-shot-react-description", verbose=True, callback_manager=manager)
        )
    tasks = [async_agent.arun(QUESTION) for async_agent in agents]
    await asyncio.gather(*tasks)
    await aiosession.close()

s = time.perf_counter()
# If running this outside of Jupyter, use asyncio.run(generate_concurrently())
await generate_concurrently()
elapsed = time.perf_counter() - s
print(f"Concurrent executed in {elapsed:0.2f} seconds.")



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner" I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner" I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner"
Observation: Rafael Nadal
Thought:
Observation: Rafael Nadal
Thought:
Observation: Rafael Nadal
Thought: I need to find out Rafael Nadal's age
Action: Search
Action Input: "Rafael Nadal age"
Observation: 36 years
Thought: I need to find out who won the US Open men's fina

## Using Tracing with Asynchronous Agents

To use tracing with async agents, you must pass in a custom `CallbackManager` with `LangChainTracer` to each agent running asynchronously. This way, you avoid collisions while the trace is being collected.

In [7]:
# To make async requests in Tools more efficient, you can pass in your own aiohttp.ClientSession, 
# but you must manually close the client session at the end of your program/event loop
aiosession = ClientSession()
tracer = LangChainTracer()
tracer.load_default_session()
manager = CallbackManager([StdOutCallbackHandler(), tracer])

# Pass the manager into the llm if you want llm calls traced.
llm = OpenAI(temperature=0, callback_manager=manager)

async_tools = load_tools(["llm-math", "serpapi"], llm=llm, aiosession=aiosession)
async_agent = initialize_agent(async_tools, llm, agent="zero-shot-react-description", verbose=True, callback_manager=manager)
await async_agent.arun(QUESTION)
await aiosession.close()



> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner"
Observation: Rafael Nadal
Thought: I need to find out Rafael Nadal's age
Action: Search
Action Input: "Rafael Nadal age"
Observation: 36 years
Thought: I need to calculate 36 raised to the 0.334 power
Action: Calculator
Action Input: 36^0.334
Observation: Answer: 3.3098250249682484

Thought: I now know the final answer
Final Answer: Rafael Nadal, aged 36, won the US Open men's final in 2019 and his age raised to the 0.334 power is 3.3098250249682484.

> Finished chain.
